# Less-Referenced Mosaic

This notebook provides a cross-section of the Less-Referenced Mosaic creation process.

# Setup

## Imports

In [ ]:
import os

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import check_random_state
import yaml

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_style('white')

In [ ]:
from night_horizons import utils, preprocess, mosaic, raster, pipelines

## Settings

In [ ]:
with open('./config.yml', "r", encoding='UTF-8') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
local_settings = {
    'mosaic_filepath': 'mosaics/referenced3.tiff',
    'overwrite': True,
    'random_state': 16849,
    'train_size': 1,
    'padding_factor': 2.,
}
settings.update(local_settings)

## Parse Settings

In [ ]:
settings['mosaic_filepath'] = os.path.join(settings['data_dir'], settings['mosaic_filepath'])

In [ ]:
for key, relpath in settings['paths_relative_to_data_dir'].items():
    settings[key] = os.path.join(settings['data_dir'], relpath)

In [ ]:
random_state = check_random_state(settings['random_state'])

In [ ]:
referenced_fps = utils.discover_data(settings['referenced_images_dir'], ['tif', 'tiff'], pattern=r'Geo\s\d+_\d.tif')

In [ ]:
palette = sns.color_palette(settings['color_palette'])

# Prepare Data
The first part is to prepare the data (AKA extract/transform/load).

## Train-Test Split

We split the data into training data (data that is georeferenced) and test data (data that is not georeferenced, or for which we don't use the georeferencing information when we're building the models).

We set the train size to some small number, because ideally the user only needs to georeference a couple of images manually.

In [ ]:
settings['train_size']

In [ ]:
fps_train, fps_test = train_test_split(referenced_fps, train_size=settings['train_size'], random_state=settings['random_state'])

## Initial, Approximate Georeferencing
We use the sensor (high-altitude balloon) positions to provide approximate georeferencing, which will be useful for saving computational time when building the unreferenced mosaic.

In [ ]:
# Get the pipeline(s)
sensor_georeference_pipeline, sensor_georeference_pipeline_y = pipelines.GeoreferencePipelines.sensor_georeference()
sensor_georeference_pipeline.set_params(nitelite__output_columns=['filepath', 'sensor_x', 'sensor_y'], sensor_georeference__passthrough=['filepath'])

In [ ]:
# Get the geo-transforms used for training
geotransforms_train = sensor_georeference_pipeline_y.fit_transform(fps_train)

In [ ]:
# Train the pipeline
sensor_georeference_pipeline.fit(
    fps_train,
    geotransforms_train,
    nitelite__img_log_fp=settings['img_log_fp'],
    nitelite__imu_log_fp=settings['imu_log_fp'],
    nitelite__gps_log_fp=settings['gps_log_fp'],
)

In [ ]:
# Get the approximate georeferences for the data we'll be testing
X_test = sensor_georeference_pipeline.predict(fps_test)

In [ ]:
# The score for the fit gives us an estimate of the error from the approximate georeferencing
first_pass_error = sensor_georeference_pipeline.score(fps_train, geotransforms_train)
padding = settings['padding_factor'] * first_pass_error

## Create the Containing Mosaic Dataset

We need to create this at this time to meet two requirements: 
1. The dataset bounds are large enough to contain all the images, not just the training images.
2. The mosaic created from the training images are saved here.

In [ ]:
# If we want to start fresh
if os.path.isfile(settings['mosaic_filepath']) and settings['overwrite']:
    os.remove(settings['mosaic_filepath'])

In [ ]:
mos = mosaic.Mosaic(settings['mosaic_filepath'], padding=first_pass_error * 2.)

In [ ]:
# The dataset is created during the fit.
mos.fit(X_test[['filepath'] + preprocess.GEOTRANSFORM_COLS])

In [ ]:
# Close the dataset so we can write to it.
mos.dataset_.FlushCache()
mos.dataset_ = None

## Create the Starting Image for the Mosaic

The starting image is a mosaic of the images that are referenced.

In [ ]:
# We use our canned pipeline for building a referenced mosaic
reffed_mosaic_pipeline = pipelines.MosaicPipelines.referenced_mosaic(settings['mosaic_filepath'])
reffed_mosaic_pipeline.set_params(mosaic__exist_ok=True)

In [ ]:
# Actual creation
reffed_mosaic_pipeline.fit_transform(fps_train)

# The Mosaic

In [ ]:
less_reffed_mosaic = mosaic.LessReferencedMosaic(filepath=settings['mosaic_filepath'], padding=padding)

In [ ]:
# First, the fit. This just gets the dataset information, since it already exists.
less_reffed_mosaic.fit(geotransforms_train)

## Determine order of iteration

In [ ]:
# We'll get the rows in the order we iterate over
# This is based on distance to the central image. 
iter_inds = less_reffed_mosaic.calc_iteration_indices(X_test)
X_iter = X_test.loc[iter_inds]
X_iter['order'] = np.arange(len(X_iter))

In [ ]:
# Let's take a look.
sp = sns.scatterplot(
    data=X_iter,
    x='x_center',
    y='y_center',
    hue='order',
)
sp.set_aspect('equal')

## First Image
We'll test the first loop in greater detail than the others.

In [ ]:
i = 0
row = X_iter.iloc[i]

In [ ]:
# The empty image at the time of the first loop
mosaic_img = less_reffed_mosaic.dataset_.ReadAsArray().transpose(1, 2, 0)

### Search Region in the Context of the Full Mosaic

In [ ]:
# Expected bounds
x_min = row['x_min'] - less_reffed_mosaic.padding
x_max = row['x_max'] + less_reffed_mosaic.padding
y_min = row['y_min'] - less_reffed_mosaic.padding
y_max = row['y_max'] + less_reffed_mosaic.padding

In [ ]:
# Convert to offset and size
x_off, y_off, x_size, y_size = less_reffed_mosaic.bounds_to_offset(x_min, x_max, y_min, y_max)

In [ ]:
fig = plt.figure(figsize=(20,10))
ax = plt.gca()

# Current mosaic
ax.imshow(mosaic_img)

# The first image location
rect = patches.Rectangle(
    (x_off, y_off),
    x_size,
    y_size,
    linewidth = 3,
    facecolor = 'none',
    edgecolor = palette[0],
)
ax.add_patch(rect)

### Search Region Image 

In [ ]:
# The existing mosaic at this location
dst_img = less_reffed_mosaic.get_image(x_min, x_max, y_min, y_max)

In [ ]:
# At this time we expect all data added to the mosaic to be within the bounds of the search region
assert dst_img.sum() == mosaic_img.sum()

This is plotted below with matched features.

In [ ]:
# Here's a zoomed in version so we know what we should be looking at.
row_train = geotransforms_train.iloc[0]
dst_img_zoom = less_reffed_mosaic.get_image(
    row_train['x_min'], row_train['x_max'],
    row_train['y_min'], row_train['y_max']
)
plt.imshow(dst_img_zoom)

### New Image

In [ ]:
src_img = utils.load_image(
    row['filepath'],
    dtype=less_reffed_mosaic.dtype,
)

This is plotted below with matched features.

### Feature Matching

In [ ]:
# Get and validate the transform predicted from feature matching
M, info = utils.calc_warp_transform(src_img, dst_img)
assert utils.validate_warp_transform(M, less_reffed_mosaic.homography_det_min)

In [ ]:
# Inspect relationship
mask = info['mask'].reshape(info['mask'].size).astype(bool)
valid_src_pts = info['src_pts'].reshape((mask.size, 2))[mask]
valid_dst_pts = info['dst_pts'].reshape((mask.size, 2))[mask]

In [ ]:
subplot_mosaic = [['dst_img', 'src_img']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['dst_img']
ax.imshow(dst_img)

ax = ax_dict['src_img']
ax.imshow(src_img)

for i in range(valid_src_pts.shape[0]):

    con = patches.ConnectionPatch(
        xyA=valid_dst_pts[i],
        xyB=valid_src_pts[i],
        coordsA='data',
        coordsB='data',
        axesA=ax_dict['dst_img'],
        axesB=ax_dict['src_img'],
        color=palette[1],
        linewidth=3,
    )
    ax.add_artist(con)

The feature matching above should look pretty good. It does as I'm writing this.

### Warp the Source Image

In [ ]:
# Warp the image being fit
warped_img = utils.warp_image(src_img, dst_img, M)

In [ ]:
plt.imshow(warped_img)

In [ ]:
# The warped image should have the same dimensions as the dst img
assert warped_img.shape[:2] == dst_img.shape[:2]

### Blend the images

In [ ]:
blended_img = less_reffed_mosaic.blend_images(
    src_img=warped_img,
    dst_img=dst_img,
)

In [ ]:
plt.imshow(blended_img)

### Save and look at the mosaic

In [ ]:
less_reffed_mosaic.save_image(blended_img, x_min, x_max, y_min, y_max)

In [ ]:
# Get the region of just the first image for comparison from before
dst_img_zoom_after = less_reffed_mosaic.get_image(
    row_train['x_min'], row_train['x_max'],
    row_train['y_min'], row_train['y_max']
)

In [ ]:
# More content should have been added
assert dst_img_zoom_after.sum() > dst_img_zoom.sum()

In [ ]:
subplot_mosaic = [['before', 'after']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['before']
ax.imshow(dst_img_zoom)

ax = ax_dict['after']
ax.imshow(dst_img_zoom_after)

## Next Image

In [ ]:
i = 1
row = X_iter.iloc[i]

In [ ]:
# Expected bounds
x_min = row['x_min'] - less_reffed_mosaic.padding
x_max = row['x_max'] + less_reffed_mosaic.padding
y_min = row['y_min'] - less_reffed_mosaic.padding
y_max = row['y_max'] + less_reffed_mosaic.padding

In [ ]:
# The existing mosaic at this location
dst_img = less_reffed_mosaic.get_image(x_min, x_max, y_min, y_max)

In [ ]:
src_img = utils.load_image(
    row['filepath'],
    dtype=less_reffed_mosaic.dtype,
)

In [ ]:
# Get and validate the transform predicted from feature matching
M, info = utils.calc_warp_transform(src_img, dst_img)
# assert utils.validate_warp_transform(M, less_reffed_mosaic.homography_det_min)

In [ ]:
# Inspect relationship
mask = info['mask'].reshape(info['mask'].size).astype(bool)
valid_src_pts = info['src_pts'].reshape((mask.size, 2))[mask]
valid_dst_pts = info['dst_pts'].reshape((mask.size, 2))[mask]

In [ ]:
subplot_mosaic = [['dst_img', 'src_img']]
fig = plt.figure(figsize=(20,10))
ax_dict = fig.subplot_mosaic(subplot_mosaic)

ax = ax_dict['dst_img']
ax.imshow(dst_img)

ax = ax_dict['src_img']
ax.imshow(src_img)

for i in range(valid_src_pts.shape[0]):

    con = patches.ConnectionPatch(
        xyA=valid_dst_pts[i],
        xyB=valid_src_pts[i],
        coordsA='data',
        coordsB='data',
        axesA=ax_dict['dst_img'],
        axesB=ax_dict['src_img'],
        color=palette[1],
        linewidth=3,
    )
    ax.add_artist(con)

This is clearly of a different area, probably because it's from a different camera, so the sensor approximation is not-so-great.
I think I'll have to include camera number in iteration order.

In [ ]:
src_img = utils.load_image(
    row['filepath'],
    dtype=less_reffed_mosaic.dtype,
)

In [ ]:
return_code, info = less_reffed_mosaic.incorporate_image(row)

## Full Mosaic

Now that we've checked the process, we'll do the full loop.

In [ ]:
# Then create the referenced mosaic
# less_reffed_mosaic.predict(X_test[['filepath'] + preprocess.GEOTRANSFORM_COLS])